# Сжатие сигнала путем свертки с линейно-частотно модулированным (ЛЧМ) сигналом <a id="intro">

## 1. Описание проекта

**Целью данного проекта** является определение ключевой характеристики изделия - длительности сжатого сигнала в наносекундах (нс). В общем случае, чем меньшую длительность имеет сжатый сигнал, тем выше пространственное разрешение (точность определения координаты объекта в пространстве) радиолокационной системы.

**План работы:**
1. Считать данные из файлов контрольно-измерительного оборудования.
1. Преобразовать комплексные числа в децибелы (дБ).
1. Провести свертку сигналов и обратное Фурье-преобразование для перехода из частотной во временную область.
1. Определить положение пика (индекс) амплитудно-частотной характеристики
1. Провести линейную интерполяцию соседних к данному точек
1. Рассчитать интересующую длительность импульса по уровню -6 дБ.

## Содержание <a id="contents"> 

1. [Описание проекта](#intro)  
2. [Чтение данных формирующих и сжимающих линий задержек](#reading)  
3. [Свертка сигналов и обратное Фурье-преобразование](#convolution)      
4. [Определение длительности сжатого импульса](#calculation)

## 2. Чтение данных формирующих и сжимающих линий задержек<a id="reading">  
[к содержанию](#contents)

In [1]:
from pylab import *
import pandas as pd
import numpy as np
from pdf_build import pdf_report
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
from matplotlib import pyplot as plt
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
%matplotlib inline
from scipy.interpolate import interp1d
from scipy.signal import chirp

Указываем имена обрабатываемых S2P-файлов:

In [2]:
UP_filename = 'sample5(UP)_after_welding.s2p' # для ДАЛЗ110В
DN_filename = 'sample6(DN)_after_welding.s2p' # для ДАЛЗ110У

Читаем данные из файла для ВОСХОДЯЩИХ/ФОРМИРУЮЩИХ линий задержек:

In [3]:
freq = []
UP_S21 = []
UP_S21_ABS = []
UP_S21_dB = []
with open(UP_filename) as data:
    for line in range(5): # пропускаем первые пять строк файла
        next(data)
    for line in data:
        freq.append(float(line.split()[0]))  # берем значения частот из 0-го столбца
        UP_S21.append(complex(float(line.split()[3]), float(line.split()[4]))) # берем значения S21 из 3 и 4-го столбца
        UP_S21_ABS.append(abs(complex(float(line.split()[3]), float(line.split()[4])))) # складываем как комплексное число по модулю
        UP_S21_dB.append(round((20 * math.log10(abs(complex(float(line.split()[3]), float(line.split()[4]))))), 2))
print("ДАЛЗ UP в дБ:", UP_S21_dB[0:10], 'и так далее...')

ДАЛЗ UP в дБ: [-69.43, -69.15, -69.47, -69.41, -69.27, -69.48, -69.33, -68.96, -69.82, -69.53] и так далее...


Читаем данные из s2p-файла для УБЫВАЮЩИХ/СЖИМАЮЩИХ линий задержек:

In [4]:
DN_S21 = []
DN_S21_ABS = []
DN_S21_dB = []
with open(DN_filename) as data:
    for line in range(5):
        next(data)
    for line in data:
        DN_S21.append(complex(float(line.split()[3]), float(line.split()[4])))
        DN_S21_ABS.append(abs(complex(float(line.split()[3]), float(line.split()[4]))))
        DN_S21_dB.append(round((20 * math.log10(abs(complex(float(line.split()[3]), float(line.split()[4]))))), 2))
print("ДАЛЗ DN в дБ:", DN_S21_dB[0:10], 'и так далее...')

ДАЛЗ DN в дБ: [-67.12, -67.15, -67.06, -67.06, -67.31, -67.41, -66.93, -67.24, -67.26, -67.06] и так далее...


## 3. Свертка сигналов и обратное Фурье-преобразование<a id="convolution">
[к содержанию](#contents)

Определяем частотный диапазон (span) из s2p-файла:

In [5]:
span = freq[-1] - freq[0]
print('Частотный диапазон (span) =', span * 1E-6, 'МГц')

Частотный диапазон (span) = 20.0 МГц


Делаем свертку формирующего и сжимающего сигналов в частотной области как произведение комплексных чисел:

In [6]:
result = np.array(UP_S21) * np.array(DN_S21)

Проводим обратное Фурье-преобразование из частотной во временную область, чтобы определить длительность сжатого импульса:

In [7]:
result_ifft = np.fft.ifft(result)
# можно брать меньшее количество элементов массива
# http://old.pynsk.ru/posts/2015/Nov/09/matematika-v-python-preobrazovanie-fure/#.XFfnh2lwm7T

Переводим спектр из значений по модули в дБ:

In [8]:
result_ifft_dB = []
for i in range(len(result_ifft)):
    result_ifft_dB.append(round((20 * math.log10(abs(result_ifft[i]))), 2))

result_ifft_dB[:10]

[-137.24,
 -147.05,
 -159.46,
 -154.88,
 -165.4,
 -174.82,
 -169.61,
 -176.01,
 -170.38,
 -179.81]

Считаем временной шаг для данного частотного диапазона и создаем шкалу времени в наносекундах (коэф. 1E9):

In [9]:
# чем больше span на R&S берем, тем мельче временной шаг, умножая его на количество точек получаем
time_step = round(1/span, 11) 

In [10]:
# временной интервал, который можем измерить во времени, т.е. чем больше span, тем хуже для timedomain области
time_list = []
for i in range(len(freq)):
    time_list.append(round((time_step * i * 1E9), 2))
print("Временной шаг (1/span): ", time_step * 1E9, "нс")
print("Общее время: ", round(time_step * 1E6 * (len(freq) - 1)), "мкс") # умножаем на 1Е6, чтобы результат был в микросекундах
print('---------------------------------')
print("Временной лист для оси X: ", time_list[:10], "и так далее, общее количество точек:", len(time_list), "шт.")
print('---------------------------------')
print("Значения в dB для оси Y: ", result_ifft_dB[:10],
      "и так далее, общее количество точек:", len(result_ifft_dB), "шт.")

Временной шаг (1/span):  50.0 нс
Общее время:  5000 мкс
---------------------------------
Временной лист для оси X:  [0.0, 50.0, 100.0, 150.0, 200.0, 250.0, 300.0, 350.0, 400.0, 450.0] и так далее, общее количество точек: 100001 шт.
---------------------------------
Значения в dB для оси Y:  [-137.24, -147.05, -159.46, -154.88, -165.4, -174.82, -169.61, -176.01, -170.38, -179.81] и так далее, общее количество точек: 100001 шт.


## 3. Определение длительности сжатого импульса<a id="calculation">
[к содержанию](#contents)

Находим максимум и его индекс:

In [11]:
maximum_dB_value = max(result_ifft_dB)
print("Максимум характеристики: ", max(result_ifft_dB), "дБ")

for i in range(len(result_ifft_dB)):
    if result_ifft_dB[i] == maximum_dB_value:
        maximum_dB_index = i

Максимум характеристики:  -76.87 дБ


Делаем линейную интерполяцию на вершине пика АЧХ для точного определения длительности по заданному уровню:

In [12]:
time_list_interp = np.linspace(time_list[maximum_dB_index - 5], time_list[maximum_dB_index + 5], num=10001)
result_ifft_dB_interp = np.interp(time_list_interp, time_list[:2000], result_ifft_dB[:2000])

Находим индекс максимуму характеристики в интерполированном ряде:

In [13]:
maximum_dB_value_interp = max(result_ifft_dB_interp)
for i in range(len(result_ifft_dB_interp)):
    if result_ifft_dB_interp[i] == maximum_dB_value_interp:
        maximum_dB_index = i

Считаем длительность по измеренным значениям ряду по уровню -6дБ (грубая оценка):

In [14]:
i = 0
time_list_level_6dB = []
for item in result_ifft_dB:
    if item > maximum_dB_value - 6:
        time_list_level_6dB.append(time_list[i])
    i += 1
print("Длительность сжатого сигнала (грубо): ",
      round((time_list_level_6dB[-1] - time_list_level_6dB[0]), 2), "нс")

Длительность сжатого сигнала (грубо):  200.0 нс


Считаем длительность по интерполированному ряду по уровню -6дБ (точное значение):

In [15]:
i = 0
time_list_level_6dB_interp = []
for item in result_ifft_dB_interp:
    if item > maximum_dB_value - 6:
        time_list_level_6dB_interp.append(time_list_interp[i])
    i += 1
print("Длительность сжатого сигнала (точно): ",
      round((time_list_level_6dB_interp[-1] - time_list_level_6dB_interp[0]), 2), "нс")

Длительность сжатого сигнала (точно):  247.75 нс


Строим результат свертки в дБ формате:

In [16]:
time_list = pd.DataFrame(time_list[:2000], columns=['time']).copy()
result_ifft_dB = pd.DataFrame(result_ifft_dB[:2000], columns=['signal']).copy()

In [17]:
result_time_domain = time_list.join(result_ifft_dB)
result_time_domain.columns = ['time, ns', 'signal, dB']
result_time_domain.head()

,"time, ns","signal, dB"
0,0.0,-137.24
1,50.0,-147.05
2,100.0,-159.46
3,150.0,-154.88
4,200.0,-165.40


In [18]:
plt.plot(result_time_domain.time, result_time_domain.signal, 'o', markersize=2)

In [ ]:
plt.plot(result_time_domain.time, result_time_domain.signal, 'o', markersize=2)

plt.show()

In [ ]:
# plt.figure(figsize=(10,10))
# plt.plot(time_list[:2000], result_ifft_dB[:2000], 'o', markersize=2)
# plt.plot(time_list_interp, result_ifft_dB_interp, '-')
# plt.xlabel('Время, нc')
# plt.ylabel('Затухание, дБ')
# plt.title('Сжатый сигнал')
# plt.grid(True)
# plt.text(36900, maximum_dB_value + 1, maximum_dB_value)
# #plt.savefig('convolution.png')
# plt.show()

# для использования pdf_build.py
# picture = ['test.png']
# f = pdf_report(picture, 'report', 'test_picture.pdf')